In [1]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools
import random
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook

import sys
sys.path.append('/home/ajhnam/sudoku/src/sudoku')

from board import Board
from grid_string import GridString, read_solutions_file
from shuffler import Shuffler
from shuffled_grid import ShuffledGrid
from solutions import Solutions
import utils

In [2]:
# set random seed to 0
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.set_default_tensor_type('torch.DoubleTensor')
device = 1

In [3]:
a = list(range(10))
random.shuffle(a)
print(a)

[7, 8, 1, 5, 3, 4, 2, 0, 9, 6]


In [3]:
filename = '/home/ajhnam/sudoku/data/shuffled_puzzles.txt'
with open(filename) as f:
    lines = f.read().splitlines()
puzzles = {}
for line in lines:
    puzzle, solution = line.split(',')
    puzzles[GridString(puzzle)] = GridString(solution)
puzzle_keys = list(puzzles.keys())
random.shuffle(puzzle_keys)

In [4]:
def determine_edges(dim_x, dim_y):
    """
    Returns a 2-d array of (max_digit**2, n) where the i_th entry is a list of
        other cells' indices that cell i shares a house with
    """
    max_digit = dim_x*dim_y
    edges = []
    for row in range(max_digit):
        row_edges = []
        for col in range(max_digit):
            # row & column
            col_edges = {(row, i) for i in range(max_digit)}
            col_edges |= {(i, col) for i in range(max_digit)}
            
            # box
            x_min = (row // dim_x) * dim_x
            y_min = (col // dim_y) * dim_y
            col_edges |= set(itertools.product(range(x_min, x_min+dim_x), range(y_min, y_min+dim_y)))
            
            # removing self
            col_edges -= {(row, col)}
            col_edges = [row*max_digit + col for row, col in col_edges]
            row_edges.append(sorted(col_edges))
        edges.append(row_edges)
    edges = torch.tensor(edges)
    shape = edges.shape
    return edges.reshape(max_digit**2, shape[2])

def encode_input(grid_string: GridString):
    return torch.tensor(list(grid_string.traverse_grid()))

def encode_output(grid_string: GridString):
    return torch.tensor(list(grid_string.traverse_grid())) - 1

In [5]:
class MLP(nn.Module):
    def __init__(self, layer_sizes):
        super(MLP, self).__init__()
        self.layer_sizes = layer_sizes
        
        self.layers = nn.ModuleList()
        self.nonlinear = nn.ReLU()
        
        prev_layer_size = self.layer_sizes[0]
        for size in self.layer_sizes[1:]:
            self.layers.append(nn.Linear(prev_layer_size, size))
            prev_layer_size = size

    def forward(self, X):
        vector = X
        for layer in self.layers[:-1]:
#             vector = layer(vector)
            vector = self.nonlinear(layer(vector))
        return self.layers[-1](vector)

class RRN(nn.Module):
    def __init__(self, dim_x, dim_y, embed_size=16, hidden_layer_size=96):
        super(RRN, self).__init__()
        self.max_digit = dim_x * dim_y
        self.embed_size = embed_size
        self.hidden_layer_size = hidden_layer_size
        
        self.edges = determine_edges(dim_x, dim_y)


        self.embed_layer = nn.Embedding(self.max_digit+1, self.embed_size)
        self.input_mlp = MLP([self.embed_size,
                              self.hidden_layer_size,
                              self.hidden_layer_size,
                              self.hidden_layer_size])
        
        self.f = MLP([2*self.hidden_layer_size,
                      self.hidden_layer_size,
                      self.hidden_layer_size,
                      self.hidden_layer_size])
        self.g_mlp = MLP([2*self.hidden_layer_size,
                      self.hidden_layer_size,
                      self.hidden_layer_size,
                      self.hidden_layer_size])
        self.g_lstm = nn.LSTM(self.hidden_layer_size, self.hidden_layer_size)
        self.r = MLP([self.hidden_layer_size,
                      self.hidden_layer_size,
                      self.hidden_layer_size,
                      self.max_digit])
    
    def compute_messages(self, H):
        messages = torch.zeros(H.shape)
        batch_size = H.shape[0]
        num_nodes = H.shape[1]
        for puzzle_index in range(batch_size): # for puzzle in batch
            messages[puzzle_index] = torch.tensor([torch.sum(H[puzzle_index][self.edges[n]]) for n in range(num_nodes)])
        return messages
                    

    def forward(self, grids, iters):
        batch_size = len(grids)
        num_nodes = self.max_digit**2
        edges_per_nodes = self.edges.shape[1]
        
        embeddings = self.embed_layer(grids)
        X = self.input_mlp(embeddings)
        H = torch.tensor(X).cuda(device)
        g_lstm_h = H.reshape(1, batch_size*num_nodes, self.hidden_layer_size)
        g_lstm_c = torch.randn(1, batch_size*num_nodes, self.hidden_layer_size).cuda(device)
        
        outputs = []
        for i in range(iters):
            M = torch.zeros(batch_size, self.max_digit**2, self.hidden_layer_size).cuda(device)
            for node in range(num_nodes):
                msgs = torch.cat([self.f(torch.cat([H[:,node,:], H[:,other,:]], dim=1)) for other in self.edges[node]])
                msgs = msgs.reshape(edges_per_nodes, batch_size, self.hidden_layer_size).permute(1,0,2)
                M[:,node,:] = torch.sum(msgs, dim=1)
            
            input_to_g_lstm = self.g_mlp(torch.cat([X, M], dim=2)).reshape(1, batch_size*num_nodes, self.hidden_layer_size)
            
            _, (g_lstm_h, g_lstm_c) = self.g_lstm(input_to_g_lstm, (g_lstm_h, g_lstm_c))
            H = g_lstm_h.reshape(H.shape)
            output = self.r(H)
            
            outputs.append(output)
                
        return outputs

In [6]:
train_n = 1000
train_puzzles = puzzle_keys[0:train_n]

train_solutions = [puzzles[p] for p in train_puzzles]

max_digit = train_puzzles[0].max_digit
num_cells = max_digit**2
cell_vec_dim = max_digit + 1
train_x = torch.cat([encode_input(p) for p in train_puzzles]).reshape(train_n, num_cells).cuda(device)
train_y = torch.cat([encode_output(p) for p in train_solutions]).reshape(train_n, num_cells).cuda(device)

In [7]:
model = RRN( dim_x=2, dim_y=2, embed_size=6, hidden_layer_size=32).cuda(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [8]:
def closure():
    optimizer.zero_grad()
    predictions = [p.permute(0,2,1) for p in model(train_x, 32)]
    loss = sum([F.cross_entropy(p, train_y) for p in predictions])
    loss.backward()
    return loss

for i in tqdm_notebook(range(250)):
    print(optimizer.step(closure))

tensor(44.4890, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.4717, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.4548, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.4397, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.4235, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.4076, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3924, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3800, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3691, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3602, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3502, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3387, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3276, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3153, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.3026, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.2935, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.2757, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(44.2490

tensor(21.5902, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(21.3358, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(21.1536, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(20.8570, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(20.7575, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(20.4796, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(20.3180, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(20.2598, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(19.7587, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(19.5186, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(18.8968, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(18.3984, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(18.1597, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(17.8333, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(17.8072, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(17.0278, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(16.7682, device='cuda:1', grad_fn=<ThAddBackward>)
tensor(16.4474

In [12]:
torch.save(model.state_dict(), "./250.pth")

In [7]:
model2 = RRN( dim_x=2, dim_y=2, embed_size=6, hidden_layer_size=32).cuda(device)
model2.load_state_dict(torch.load("./250.pth"), strict=False)
model2.eval()

RRN(
  (embed_layer): Embedding(5, 6)
  (input_mlp): MLP(
    (layers): ModuleList(
      (0): Linear(in_features=6, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
    (nonlinear): ReLU()
  )
  (f): MLP(
    (layers): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
    (nonlinear): ReLU()
  )
  (g_mlp): MLP(
    (layers): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
    (nonlinear): ReLU()
  )
  (g_lstm): LSTM(32, 32)
  (r): MLP(
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, b

In [8]:
test_n = 1000
test_puzzles = puzzle_keys[train_n:train_n+test_n]
test_solutions = [puzzles[p] for p in test_puzzles]

test_x = torch.cat([encode_input(p) for p in test_puzzles]).reshape(test_n, num_cells).cuda(device)
test_y = torch.cat([encode_output(p) for p in test_solutions]).reshape(test_n, num_cells).cuda(device)

In [9]:
softmax = nn.Softmax(dim=2)
predictions = [softmax(p) for p in model2(test_x, 32)]
output = predictions[-1].argmax(dim=2)

In [16]:
optimizer = optim.Adam(model2.parameters(), lr=1e-3, weight_decay=1e-4)

def closure():
    optimizer.zero_grad()
    predictions = [p.permute(0,2,1) for p in model(train_x, 32)]
    loss = sum([F.cross_entropy(p, train_y) for p in predictions])
    loss.backward()
    return loss

for i in tqdm_notebook(range(250)):
    print(optimizer.step(closure))

tensor([4, 3, 1, 2, 2, 1, 0, 3, 1, 2, 3, 4, 0, 4, 2, 1], device='cuda:1')

In [14]:
test_y[9]

tensor([3, 2, 0, 1, 1, 0, 3, 2, 0, 1, 2, 3, 2, 3, 1, 0], device='cuda:1')